In [1]:
from model import *
from data import *

C:\SoftWare\Anaconda2\envs\python3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Train your Unet with membrane data
membrane data is in folder membrane/, it is a binary classification task.

The input shape of image and mask are the same :(batch_size,rows,cols,channel = 1)

### Train with data generator

In [2]:
# 데이터 부풀리기
# 1) rotation_range : 지정된 각도 범위 내에서 임의로 원본 이미지를 회전함.
# 2) width_shift_range : 지정된 수평방향 이동 범위 내에서 임의로 원본 이미지를 이동시킴. 수치는 전체 넓이의 비율로 나타냄
# 3) height_shift_range : 지정된 수직방향 이동 범위 내에서 임의로 원본 이미지를 이동시킴. 수치는 전체 높이의 비율로 나타냄.
# 4) shear_range : 밀림 강도 범위 내에서 임의로 원본 이미지를 변형시킴. 시계 반대 방향으로 밀림 강도를 라디안으로 나타냄.
# 5) zoom_range : 지정된 확대/축소 범위 내에서 임의로 원본 이미지를 확대/축소함. '1-수치'부터 '1+수치'사이 범위로 확대/축소함.
# 6) horizontal_flip : 수평방향으로 뒤집기를 함.
# 7) fill_mode : 이미지를 회전, 이동하거나 축소할 때 생기는 공간을 채우는 방식
# fill_mode='nearest' : aaaaaaaa|abcd|dddddddd
data_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')

# Generator : 데이터셋 생성
# trainGenerator : 트레이닝 데이터셋 생성
# 2 : batch size
# save_to_dir : 저장 경로 지정
myGene = trainGenerator(2,'data/membrane/train','image','label',data_gen_args,save_to_dir = None)

model = unet()

# ModelCheckpoint :  매 epoch마다 모델의 가중치를 저장하도록함.
# ModelCheckpoint(filepath, monitor, verbose, save_best_only)
# 1) filepath : 인자에서 실제 가중치를 저장할 경로를 지정함.
# 2) monitor : 관찰하고자 하는 항목
# 3) verbose : 1일 시 학습을 진행하는동안 가중치 파일이 업데이트 될 때마다 텍스트를 출력함.
# 4) save_best_only : True시 validation set에서 가장 정확도가 높았던 시행에서의 가중치만을 저장함.
model_checkpoint = ModelCheckpoint('unet_membrane.hdf5', monitor='loss',verbose=1, save_best_only=True)

# Generator로 생성된 배치로 학습시킬 경우 fit_generator() 함수를 사용함.
# model.fit_generator(첫번째인자, steps_per_epoch, epochs, callbacks)
# 1) 첫 번째 인자 : 훈련데이터셋을 제공할 generator 지정.
# 2) steps_per_epoch : 한 epoch에 사용한 스텝 수를 지정. (훈련 샘플 수/배치 사이즈)
# 3) epochs : 전체 훈련 데이터셋에 대해 학습 반복 횟수를 지정.
# 4) callbacks : 콜백할 위치 지정.
model.fit_generator(myGene,steps_per_epoch=2000,epochs=5,callbacks=[model_checkpoint])

C:\Users\xuhaozhi\Documents\Study\unet\model.py:34: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  merge6 = merge([drop4,up6], mode = 'concat', concat_axis = 3)
C:\SoftWare\Anaconda2\envs\python3\lib\site-packages\keras\legacy\layers.py:465: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
C:\Users\xuhaozhi\Documents\Study\unet\model.py:39: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  merge7 = merge([conv3,up7], mode = 'concat', concat_axis = 3)
C:\Users\xuhaozhi\Documents\Study\unet\model.py:44: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `

Epoch 1/5
Found 30 images belonging to 1 classes.
Found 30 images belonging to 1 classes.
2000/2000 [==============================] - 538s 269ms/step - loss: 0.1951 - acc: 0.9140

Epoch 00001: loss improved from inf to 0.19511, saving model to unet_membrane.hdf5
Epoch 2/5
2000/2000 [==============================] - 535s 268ms/step - loss: 0.1210 - acc: 0.9469

Epoch 00002: loss improved from 0.19511 to 0.12099, saving model to unet_membrane.hdf5
Epoch 3/5
2000/2000 [==============================] - 534s 267ms/step - loss: 0.0926 - acc: 0.9596

Epoch 00003: loss improved from 0.12099 to 0.09256, saving model to unet_membrane.hdf5
Epoch 4/5
2000/2000 [==============================] - 534s 267ms/step - loss: 0.0788 - acc: 0.9656

Epoch 00004: loss improved from 0.09256 to 0.07882, saving model to unet_membrane.hdf5
Epoch 5/5
2000/2000 [==============================] - 534s 267ms/step - loss: 0.0707 - acc: 0.9691

Epoch 00005: loss improved from 0.07882 to 0.07074, saving model to une

### Train with npy file

In [ ]:
# 트레이닝 데이터와 트레이닝 마스크 데이터 경로
imgs_train,imgs_mask_train = geneTrainNpy("data/membrane/train/aug/","data/membrane/train/aug/")

# model.fit(X_train, y_train, batch_size, nb_epoch, verbose, validation_split, shuffle, callbacks)
# 1) X_train : 훈련 데이터
# 2) y_train : 지도 학습 관점에서 레이블 데이터
# 3) batch_size : 배치의 크기. keras에서 배치의 기본값은 32.
# 4) nb_epoch : 학습 epoch 횟수
# 5) verbose : 학습 중 출력되는 문구 설정 ( 0 : 아무 것도 출력하지 않음. / 2 : 에포크 횟수당 한 줄씩 출력함. / 1 : 훈련의 진행도를 보여주는 진행 막대를 보여줌.)
# 6) validation_split : X_train과 y_train에서 일정 비율을 분리하여 이를 검증 데이터로 사용함. 훈련 자체에는 반영되지 않고 훈련 과정을 지켜보기 위한 용도로 사용됨. (예: 0.2면 훈련 데이터의 20%를 검증 데이터로 사용)
# 7) shuffle : 각 epoch마다 샘플을 섞을지 여부를 나타냄.
# 8) callbacks : 콜백할 위치 지정.
model.fit(imgs_train, imgs_mask_train, batch_size=2, nb_epoch=10, verbose=1,validation_split=0.2, shuffle=True, callbacks=[model_checkpoint]

### test your model and save predicted results

In [3]:
# test할 데이터셋 경로
testGene = testGenerator("data/membrane/test")

model = unet()

# 가중치 불러오기
model.load_weights("unet_membrane.hdf5")

# predict_generator : 데이터 생성기로부터 입력 샘플에 대한 예측을 생성함.
# predict_generator(self, max_queue_size, verbose)
# 1) max_queue_size : 생성기 큐의 최대 크기
# 2) verbose : 1일 시 상세 정보 모드를 업데이트 될 때마다 텍스트로 출력함.
results = model.predict_generator(testGene,30,verbose=1)

# 결과값 저장
saveResult("data/membrane/test",results)

C:\Users\xuhaozhi\Documents\Study\unet\model.py:34: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  merge6 = merge([drop4,up6], mode = 'concat', concat_axis = 3)
C:\SoftWare\Anaconda2\envs\python3\lib\site-packages\keras\legacy\layers.py:465: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
C:\Users\xuhaozhi\Documents\Study\unet\model.py:39: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  merge7 = merge([conv3,up7], mode = 'concat', concat_axis = 3)
C:\Users\xuhaozhi\Documents\Study\unet\model.py:44: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `

 1/30 [>.............................] - ETA: 4s

C:\SoftWare\Anaconda2\envs\python3\lib\site-packages\skimage\transform\_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\SoftWare\Anaconda2\envs\python3\lib\site-packages\skimage\transform\_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


30/30 [==============================] - 1s 47ms/step


C:\SoftWare\Anaconda2\envs\python3\lib\site-packages\skimage\util\dtype.py:130: UserWarning: Possible precision loss when converting from float32 to uint16
  .format(dtypeobj_in, dtypeobj_out))
